In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### 1. Dataloader

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

train_dataset = torchvision.datasets.CIFAR10(root = '../../data', 
                                           train = True, 
                                           transform = transform,
                                           download = True
                                          )

test_dataset = torchvision.datasets.CIFAR10(root = '../../data', 
                                           train = False, 
                                           transform = transform,
                                           download = True
                                          )

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = False)

### 2. Model

In [6]:
class CNN(nn.Module):
    def __init__(self, num_classes = 10):
        super(CNN, self).__init__()
        self.conv_1 = nn.Conv2d(3, 32, 3)
        self.conv_2 = nn.Conv2d(32, 64, 3)
        self.conv_3 = nn.Conv2d(64, 64, 3)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.fc_1 = nn.Linear(64 * 4 * 4, 64)
        self.fc_2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        # input = N, 3, 32, 32
        x = self.conv_1(x)
        x = F.relu(x)
        x = self.max_pool(x)
#         print(x.shape)
        x = self.conv_2(x)
        x = F.relu(x)
        x = self.max_pool(x)
        
        x = self.conv_3(x)
        x = F.relu(x)
#         x = self.max_pool(x)

        x = torch.flatten(x, 1)
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        return x

In [8]:
model = CNN().to(device)

### 3. Training

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [11]:
n_total_steps = len(train_loader)
for epoch in range(5):
    print('Starting epoch:', epoch)
    running_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item()
    print(f'Epoch: {epoch}, loss: {running_loss / n_total_steps}')
print('finished training')

Starting epoch: 0
Epoch: 0, loss: 1.4878485550197058
Starting epoch: 1
Epoch: 1, loss: 1.1175944060754563
Starting epoch: 2
Epoch: 2, loss: 0.9578785018629549
Starting epoch: 3
Epoch: 3, loss: 0.8576310885044068
Starting epoch: 4
Epoch: 4, loss: 0.781754967919238
finished training


### 4. Saving Model

In [ ]:
model_save_path = '../../model/cnn_cifar10.pth')
torch.save(model.state_dict(), model_save_path) 

In [ ]:
model = CNN()
model.load_state_dict(torch.load(model_save_path))
model.to(device)
model.eval()

### 5. Evaluate

In [12]:
with torch.no_grad():
    correct = 0
    samples = len(test_loader.dataset)
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
    print(correct/samples)

0.7038
